<h2> User interface for visualization located at the bottom of file

In [3]:
import scipy
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter, binary_closing, binary_opening, label
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
import cmocean
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [4]:
def convertDiffuseFile(fileInName):   
    fileContents = scipy.io.loadmat(fileInName)
    covisDict = {}
    covisDict['header'] = fileContents['__header__'].decode('utf-8')
    covisDict['version'] = fileContents['__version__']
    covisDict['globals'] = fileContents['__globals__']
    covis = fileContents['covis'][0][0]
    for name in covis.dtype.names:
        covisDict[name] = covis[name]
    
    print(f'{fileInName} has been converted')
    return covisDict

In [5]:
def convertImagingFile(fileInName):
    fileContents = scipy.io.loadmat(fileInName)
    covisDict = {}
    covisDict['header'] = fileContents['__header__'].decode('utf-8')
    covisDict['version'] = fileContents['__version__']
    covisDict['globals'] = fileContents['__globals__']
    covis = fileContents['imaging'][0][0]
    for name in covis.dtype.names:
        covisDict[name] = covis[name]
    
    print(f'{fileInName} has been converted')
    return covisDict

In [6]:
def readCoords2D(fileInName):
    covisDict = convertDiffuseFile(fileInName)
    wList = [w for w in covisDict['grid'][0][0][0]['w']]
    xList = [x for x in covisDict['grid'][0][0][0]['x']]
    yList = [y for y in covisDict['grid'][0][0][0]['y']]
    vList = [v for v in covisDict['grid'][0][0][0]['v']]
    coordsDict = {'xList': xList, 'yList': yList, 'vList': vList, 'wList': wList}
    return coordsDict

In [7]:
def readCoords3D(fileInName, dataVal):
    covisDict = convertImagingFile(fileInName)
    xList = [x for x in covisDict['grid'][0][0]['x']]
    yList = [y for y in covisDict['grid'][0][0]['y']]
    zList = [z for z in covisDict['grid'][0][0]['z']]
    vList = [v for v in covisDict['grid'][0][0][f'{dataVal}']]
    wList = [w for w in covisDict['grid'][0][0]['w']]
    coordsDict = {'xList': xList, 'yList': yList, 'zList': zList, 'vList': vList, 'wList': wList}
    return coordsDict

In [8]:
def readCoordsBathymetry(bathyFileInName, diffuseFileInName):
    bathyDict = convertDiffuseFile(bathyFileInName)
    diffuseDict = convertDiffuseFile(diffuseFileInName)
    xList = [x for x in bathyDict['grid'][0][0]['x']]
    yList = [y for y in bathyDict['grid'][0][0]['y']]
    zList = [z for z in bathyDict['grid'][0][0]['v']]
    vList = [v for v in diffuseDict['grid'][0][0][0]['v']]
    coordsDict = {'xList': xList, 'yList': yList, 'zList': zList, 'vList': vList}
    return coordsDict

In [9]:
def removeZeroSlices(arr, axis, coordsDict):
    red_axes = tuple(i for i in range(arr.ndim) if i != axis)
    mask = np.any(arr, axis = red_axes)
    slicing = tuple(slice(None) if i != axis else mask for i in range(arr.ndim))
    for key, value in coordsDict.items():
        coordsDict[key] = value[slicing]
    return coordsDict

In [10]:
def removeAllZeros(coordsDict):
    mask = coordsDict['vList'] > 0
    coordsDict = {key: coordsDict[key][mask] for key in coordsDict.keys()}
    return coordsDict

In [11]:
def createNDArray(coordsDict, dim):
    xmin, xmax = coordsDict['xList'].min(), coordsDict['xList'].max()
    ymin, ymax = coordsDict['yList'].min(), coordsDict['yList'].max()
    zmin, zmax = 0, 0
    minmaxDict = {'x': (xmin, xmax), 'y': (ymin, ymax)}

    if dim == 3:
        zmin, zmax = coordsDict['zList'].min(), coordsDict['zList'].max()
        minmaxDict['z'] = (zmin, zmax)
    
    dimDict = {}
    for key, value in minmaxDict.items():
        dimDict[key] = int((value[1] - value[0]) * 4 + 1)

    if dim == 2:
        for key, value in coordsDict.items():
            coordsDict[key] = value.reshape(dimDict['x'], dimDict['y'])
    elif dim == 3:
        for key, value in coordsDict.items():
            coordsDict[key] = value.reshape(dimDict['x'], dimDict['y'], dimDict['z'])

    return coordsDict

In [12]:
def createCoordsOfInterest(fileInName, opt1, opt2 = 'Ia', bathyFile = r'matFiles\covis_bathy_2019b.mat'):
	if opt1 == 2:
		coordsDict = readCoords2D(fileInName)
		coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}

		vList = coordsDict['vList']
		for i in [0, 1, 2]:
			coordsDict = removeZeroSlices(vList, i, coordsDict)

		coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

	elif opt1 == 3:
		coordsDict = readCoords3D(fileInName, opt2)
		coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}

		vList = coordsDict['vList']
		for i in [0, 1, 2]:
			coordsDict = removeZeroSlices(vList, i, coordsDict)

		coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

	elif opt1 == 4:
		coordsDict = readCoordsBathymetry(bathyFile, fileInName)
		coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}
		coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

		coordsDict['vList'] = coordsDict['vList'].reshape(121, 121)[:,:101]
		coordsDict['vList'] = np.concatenate(coordsDict['vList']).flatten()
		coordsDict['zList'] = np.nan_to_num(coordsDict['zList'])
		
		x = coordsDict['xList']
		y = coordsDict['yList']
		indices = np.lexsort((y, x))
		coordsDict = {key: coordsDict[key][indices].reshape(101, 121) for key in coordsDict.keys()}
		
	elif opt1 == 5:
		coordsDict = readCoords3D(fileInName, opt2)
		coordsDict = {key: np.array(coordsDict[key]) for key in coordsDict.keys()}

		coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}
	
	return coordsDict

In [13]:
def plotDiffuse2D(data, fileInName):
    plt.figure(figsize=(8, 6))

    data = removeAllZeros(data)
    data['vList'] = np.log10(data['vList'])
    max = np.floor(np.amax(data['vList']))
    min = np.ceil(np.amin(data['vList']))
    scatter = plt.scatter(data['xList'], data['yList'], c = data['vList'], cmap = 'viridis')

    cbar = plt.colorbar(scatter)
    cbar_ticks = np.arange(min, max + 1, 1)
    cbar.set_ticks(cbar_ticks)
    cbar.set_label('Data Values')

    plt.xlabel('East of COVIS (m)')
    plt.ylabel('North of COVIS (m)')
    plt.title(fileInName.split('\\')[-1])
    plt.show()


In [14]:
def prepareCloudData(data, cloud):
	for key, value in data.items():
		data[key] = value.flatten()
	
	if (cloud == 'Inferno'):
		xmask = (data['xList'] >= -20) & (data['xList'] <= 0)
		for key, value in data.items():
			data[key] = value[xmask]
		ymask = (data['yList'] >= -10) & (data['yList'] <= 10)
		for key, value in data.items():
			data[key] = value[ymask]
		zmask = (data['zList'] >= 0) & (data['zList'] <= 20)
		for key, value in data.items():
			data[key] = value[zmask]
	elif (cloud == 'Mushroom'):
		xmask = (data['xList'] >= -10) & (data['xList'] <= 0)
		for key, value in data.items():
			data[key] = value[xmask]
		ymask = (data['yList'] >= 0) & (data['yList'] <= 10)
		for key, value in data.items():
			data[key] = value[ymask]
		zmask = (data['zList'] >= 0) & (data['zList'] <= 15)
		for key, value in data.items():
			data[key] = value[zmask]
	else:
		print(f"Invalid cloud: {cloud}")

	values = data['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	data['vList'] = values
	
	return data

In [15]:
def plotFullImagingCloud(data, param):
	x = data['xList']
	y = data['yList']
	z = data['zList']
	values = data['vList']
	min = -1
	max = -9

	fig = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'thermal',
		isomin = min,
		isomax = max,
		surface_count = 10,
		opacity = 0.1,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig.update_layout(
		title=f'Inferno (Logarithmic scale, {param})',
		scene=dict(
				xaxis_title='East of COVIS (m)',
				yaxis_title='North of COVIS (m)',
				zaxis_title='Z Axis'
		),
		coloraxis_colorbar=dict(
				title=f'{param} Values',
				tickvals=np.linspace(min, max, num=5),
				ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	fig.show()

In [16]:
def plotDiffuseBathymetryV1(fileInName, opt2, bathyFile = r'matFiles\covis_bathy_2019b.mat'):
	coordsOfInterest = createCoordsOfInterest(fileInName, 4, bathyFile)

	z = coordsOfInterest['zList']
	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']

	xmin, xmax = -20, 20
	ymin, ymax = -30, 1
	zmin, zmax = z.min(), z.max()

	sh_0, sh_1 = z.shape
	x = np.linspace(x.min(), x.max(), sh_0)
	y = np.linspace(y.min(), y.max(), sh_1)
	xrange = x.max() - x.min()
	range_y = y.max() - y.min()
	range_z = z.max() - z.min()
	maxrange = np.max([xrange, range_y, range_z])

	fig = go.Figure(data=[go.Surface(
		z=z,
		x=x,
		y=y, 
		)]
	)

	fig.update_layout(
		title='Bathymetry', 
		scene=dict(
			aspectmode="manual",
			aspectratio=dict(
				x=xrange / maxrange,
				y=range_y / maxrange,
				z=range_z / maxrange
			),
			xaxis=dict(range=[xmin, xmax]),
			yaxis=dict(range=[ymin, ymax]),
			zaxis=dict(range=[zmin, zmax])
		)
	)

	fig.show()

In [17]:
def plotDiffuseBathymetryV2(fileInName, opt2, bathyFile = r'matFiles\covis_bathy_2019b.mat'):
    coordsOfInterest = createCoordsOfInterest(fileInName, 4, bathyFile)
    plt.figure(figsize=(12, 8))

    x = coordsOfInterest['xList']
    y = coordsOfInterest['yList']
    data = coordsOfInterest[opt2]
    
    if opt2 == 'zList':
        param = 'height'
    elif opt2 == 'vList':
        param = 'data values'
        data = np.ma.masked_where(data == 0, data)
        mask = data > 0
        data[mask] = np.log10(data[mask])
    
    plt.pcolormesh(x, y, data, shading = 'auto')
    plt.colorbar(label = param)

    plt.xlabel('East of COVIS (m)')
    plt.ylabel('North of COVIS (m)')
    plt.title(f'Pcolor Plot of Bathymetry, {param}')

    plt.show()

In [18]:
def plotFull(diffuseFileName, bathyFileName, imagingFileName, opt2):
	coordsOfInterest = createCoordsOfInterest(diffuseFileName, 4, bathyFileName)

	z = coordsOfInterest['zList']
	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']

	sh_0, sh_1 = z.shape
	x = np.linspace(x.min(), x.max(), sh_0)
	y = np.linspace(y.min(), y.max(), sh_1)
	xrange = x.max() - x.min()
	range_y = y.max() - y.min()
	range_z = z.max() - z.min()
	x = x - 13
	y = y + 13

	fig1 = go.Figure(data=[go.Surface(
		z = z,
		x = x,
		y = y, 
		colorscale = [[0, 'tan'], [1, 'tan']], 
		showscale = False,
		contours = {
            "z": {"show": True, "highlight": True, "highlightcolor": "black", "highlightwidth": 2}
		}
		)]
	)
	
	coordsOfInterest = createCoordsOfInterest(imagingFileName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()

	xmask = (coordsOfInterest['xList'] >= -20) & (coordsOfInterest['xList'] <= 0)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[xmask]
	ymask = (coordsOfInterest['yList'] >= -10) & (coordsOfInterest['yList'] <= 10)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[ymask]
	zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 20)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[zmask]

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	min = -1
	max = -9

	fig2 = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'thermal',
		isomin = min,
		isomax = max,
		surface_count = 5,
		opacity = 0.2,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig2.update_layout(
		coloraxis_colorbar=dict(
			title=f'{opt2} Values',
			x = 1.1,
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	coordsOfInterest = createCoordsOfInterest(imagingFileName, 3, opt2)
	coordsOfInterest = createNDArray(coordsOfInterest, 3)
	for key, value in coordsOfInterest.items():
		coordsOfInterest[key] = value.flatten()

	xmask = (coordsOfInterest['xList'] >= -10) & (coordsOfInterest['xList'] <= 0)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[xmask]
	ymask = (coordsOfInterest['yList'] >= 0) & (coordsOfInterest['yList'] <= 10)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[ymask]
	zmask = (coordsOfInterest['zList'] >= 0) & (coordsOfInterest['zList'] <= 15)
	for key, value in coordsOfInterest.items():
			coordsOfInterest[key] = value[zmask]

	x = coordsOfInterest['xList']
	y = coordsOfInterest['yList']
	z = coordsOfInterest['zList']
	values = coordsOfInterest['vList']
	values[values == 0] = 10e-10
	values = np.log10(values)
	
	fig3 = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'Inferno',
		isomin = -1,
		isomax = -9,
		surface_count = 5,
		opacity = .2,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	fig3.update_layout(
		coloraxis_colorbar=dict(
			title=f'{opt2} Values',
			x = 1.2,
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)

	figFinal = go.Figure(data = fig1.data + fig2.data + fig3.data)
	figFinal.update_layout(
		title=f'Plumes on Bathymetry (Logarithmic scale, {opt2})',
		scene=dict(
			xaxis_title='East of COVIS (m)',
			yaxis_title='North of COVIS (m)',
			zaxis_title='Z Axis',
			xaxis=dict(range=[-20 - 13, 20 - 13]),
			yaxis=dict(range=[-30 + 13, 1 + 13]),
			zaxis=dict(range=[-4, 20])
		)
	)
	
	figFinal.data[1].colorbar.update(title='Inferno', x=1.1)
	figFinal.data[2].colorbar.update(title='Mushroom', x=1.2)

	figFinal.show()

In [19]:
# plotFull(r'matFiles\COVIS-20230701T003002-diffuse1.mat', r'matFiles\covis_bathy_2019b.mat', r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia_filt')

In [20]:
def splatAlongAxis(fileInName, opt2, blur, axis = 'x'):
    coordsOfInterest = createCoordsOfInterest(fileInName, 3, opt2)
    coordsOfInterest = removeAllZeros(coordsOfInterest)

    if axis == 'x':
        proj_coords = (coordsOfInterest['yList'], coordsOfInterest['zList'])
        xlabel, ylabel = 'y', 'z'
    elif axis == 'y':
        proj_coords = (coordsOfInterest['xList'], coordsOfInterest['zList'])
        xlabel, ylabel = 'x', 'z'
    elif axis == 'z':
        proj_coords = (coordsOfInterest['xList'], coordsOfInterest['yList'])
        xlabel, ylabel = 'x', 'y'
    
    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])

    minCoord1 = np.min(proj_coords[0])
    maxCoord1 = np.max(proj_coords[0])
    minCoord2 = np.min(proj_coords[1])
    maxCoord2 = np.max(proj_coords[1])
    
    gridSize = 500
    grid, _, _ = np.histogram2d(proj_coords[0], proj_coords[1], bins = gridSize, 
                                range=[[minCoord1, maxCoord1], [minCoord2, maxCoord2]], weights = coordsOfInterest['vList'])
    
    splatted_image = gaussian_filter(grid, sigma = blur)
    
    plt.imshow(splatted_image.T, origin='lower', cmap='inferno', 
               extent=[minCoord1, maxCoord1, minCoord2, maxCoord2])
    plt.colorbar(label='Accumulated Value')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(f'Splatting along {axis}-axis, {opt2}')
    plt.show()

In [21]:
def plotCenterline(data, center, opt2 = "Ia_filt"):
	x = data['xList']
	y = data['yList']
	z = data['zList']
	values = data['vList']
	min = -1
	max = -9

	fig = go.Figure(data=go.Volume(
		x = x.flatten(),
		y = y.flatten(),
		z = z.flatten(),
		value = values.flatten(),
		colorscale  = 'thermal',
		isomin = min,
		isomax = max,
		surface_count = 10,
		opacity = 0.1,
		caps = dict(x_show=False, y_show=False, z_show=False)
	))

	z_levels = data['vList'].shape[2]
	maxPoints = []
	
	if (center == "max"):
		for z in range(z_levels):
			maxValue = np.max(data['vList'][:, :, z])
			maxIndex = np.argmax(data['vList'][:, :, z])
			x, y = np.unravel_index(maxIndex, data['vList'][:, :, z].shape)
			xVal = data['xList'][x, y, z]
			yVal = data['yList'][x, y, z]
			zVal = data['zList'][x, y, z]
			maxPoints.append((xVal, yVal, zVal, maxValue))

			x, y, z, v = zip(*maxPoints)
			line = go.Figure(data = go.Scatter3d(
				x=x, 
				y=y, 
				z=z,
				mode='lines+markers',
				line=dict(color='blue', width=1),
				marker=dict(size=1)
			))
	elif (center == "geom"):
		for z in range(z_levels):
			xSlice = data['xList'][:, :, z]
			ySlice = data['yList'][:, :, z]
			vSlice = data['vList'][:, :, z]
			
			valid = vSlice != -9
			
			vValid = vSlice[valid]
			xValid = xSlice[valid]
			yValid = ySlice[valid]
			
			if vValid.size == 0:
				continue
			
			vSum = np.sum(vValid)
			xWeightedMean = np.sum(xValid * vValid) / vSum
			yWeightedMean = np.sum(yValid * vValid) / vSum
			
			z_value = data['zList'][0, 0, z]
			
			maxPoints.append((xWeightedMean, yWeightedMean, z_value))

		x, y, z = zip(*maxPoints)
		line = go.Figure(data = go.Scatter3d(
			x=x, 
			y=y, 
			z=z,
			mode='lines+markers',
			line=dict(color='blue', width=2),
			marker=dict(size=1)
		))


	figFinal = go.Figure(data = fig.data + line.data)

	figFinal.update_layout(
		title=f'Inferno (Logarithmic scale, {'Ia_filt'})',
		scene=dict(
			xaxis_title='East of COVIS (m)',
			yaxis_title='North of COVIS (m)',
			zaxis_title='Z Axis'
		),
		coloraxis_colorbar=dict(
			title=f'{'Ia_filt'} Values',
			tickvals=np.linspace(min, max, num=5),
			ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
		)
	)	

	figFinal.show()

In [22]:
def labelClouds(data, structure = np.ones((3, 3, 3))):
	val = data['vList']
	binaryMask = np.where(val > -9, 1, 0)

	labeledArr, numFeatures = label(binaryMask, structure = structure)
	unique, counts = np.unique(labeledArr, return_counts=True)
	components = dict(zip(unique[1:], counts[1:])) 
	largest = sorted(components, key = components.get, reverse = True)[:1]
	largestMask = np.isin(labeledArr, largest)
	val = np.where(largestMask, data['vList'], -9).astype(np.float64)

	data['vList'] = val
	return data

In [23]:
def morphologicalClosing(data, structure = np.ones((3, 3, 3))):
    val = data['vList']
    binaryMask = np.where(val > -9, 1, 0)
    closedArr = binary_closing(binaryMask, structure = structure)
    data['vList'] = np.where(closedArr, val, -9)
    return data

In [24]:
def morphologicalOpening(data, structure = np.ones((3, 3, 3))):
    val = data['vList']
    binaryMask = np.where(val > -9, 1, 0)
    openedArr = binary_opening(binaryMask, structure = structure)
    data['vList'] = np.where(openedArr, val, -9)
    return data

In [ ]:
# Creates 2D scatterplot of diffuse data

# User-defined variables
fileInName = r'matFiles\COVIS-20230701T003002-diffuse1.mat'

# Function calls
coordsOfInterest = createCoordsOfInterest(fileInName, 2)
plotDiffuse2D(coordsOfInterest, fileInName)

In [ ]:
# Plots a volume figure of either the inferno or mushroom plumes

# User-defined variables
fileInName = r'matFiles\COVIS-20230101T000002-imaging1.mat'
cloud = 'Inferno'
param = 'Ia_filt'

# Function calls
coordsOfInterest = createCoordsOfInterest(fileInName, 3, param)
coordsOfInterest = createNDArray(coordsOfInterest, 3)
coordsOfInterest = prepareCloudData(coordsOfInterest, cloud)
plotFullImagingCloud(coordsOfInterest, param)


In [ ]:
# Plots centerline following removal of noise

# User-defined variables
fileInName = r'matFiles\COVIS-20230101T000002-imaging1.mat'
cloud = 'Inferno'
param = 'Ia_filt'
center = 'geom'
structure = np.ones((3, 3, 3))

# Function calls
coordsOfInterest = createCoordsOfInterest(fileInName, 3, param)
coordsOfInterest = createNDArray(coordsOfInterest, 3)
coordsOfInterest = prepareCloudData(coordsOfInterest, cloud)
coordsOfInterest = createNDArray(coordsOfInterest, 3)
coordsOfInterest = labelClouds(coordsOfInterest, structure)
coordsOfInterest = morphologicalClosing(coordsOfInterest, structure)
plotCenterline(coordsOfInterest, center)

In [ ]:
# Trends

# User-defined variables
directory ='matFiles'
fileInNames = [
    os.path.join(directory, file)
    for file in os.listdir(directory)
    if "imaging1" in file and file.endswith('.mat')
]
cloud = 'Inferno'
param = 'Ia_filt'
center = 'geom'
structure = np.ones((3, 3, 3))


# Function Calls
data = [createCoordsOfInterest(fileInName, 5, param) for fileInName in fileInNames]
data = [createNDArray(arr, 3) for arr in data]
data = [prepareCloudData(arr, "Inferno") for arr in data]
data = [createNDArray(arr, 3) for arr in data]
data = [labelClouds(arr, structure) for arr in data]
data = [morphologicalClosing(arr, structure) for arr in data]

averagedData = {key: np.mean([arr[key] for arr in data], axis=0) for key in data[0]}
plotCenterline(averagedData, center)